In [13]:
# runtime/integrated_pipeline.py
"""
统一的阶段2+阶段3集成入口
将护栏层(阶段2)与生成层(阶段3)完整连接
"""

from __future__ import annotations
import json
from typing import Dict, Any, List, Optional
from pathlib import Path
import sys

# 导入项目模块
try:
    from runtime.controller import run_once
    print("成功导入runtime.controller")
    from provider.generator import Generator
    print("成功导入provider.generator")
    from provider.oocChecker import OOCChecker
    print("成功导入provider.oocChecker")
    from provider.memory_store import MemoryStore
    print("成功导入provider.memory_store")
    from provider.memory_summarizer import MemorySummarizer
    print("成功导入provider.memory_summarizer")
    from provider.qwen import QwenProvider
    print("成功导入provider.qwen")
    from runtime.compile_data import compile_all, CACHE_DIR, CACHE_FILE
    print("成功导入所有模块")
except ImportError:
    # 回退到相对导入
    try:
        from .controller import run_once
        from qwen import QwenProvider
        from generator import Generator
        from oocChecker import OOCChecker
        from memory_store import MemoryStore
        from memory_summarizer import MemorySummarizer
        from compile_data import compile_all, CACHE_DIR, CACHE_FILE
    except ImportError:
        print("警告：无法导入所有模块，请检查项目结构")

# 每次都重新加载以上导入的模块，确保使用最新代码
%load_ext autoreload
%autoreload 2

成功导入runtime.controller
成功导入provider.generator
成功导入provider.oocChecker
成功导入provider.memory_store
成功导入provider.memory_summarizer
成功导入provider.qwen
成功导入所有模块
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
class IntegratedPipeline:
    """集成阶段2和阶段3的完整对话管道"""
    
    def __init__(self, api_key: Optional[str] = None, auto_compile: bool = True):
        """
        初始化集成管道
        
        Args:
            api_key: Qwen API密钥，如果为None则从环境变量读取
            auto_compile: 是否自动编译数据文件
        """
        # 确保缓存目录存在
        CACHE_DIR.mkdir(parents=True, exist_ok=True)
        
        # 自动编译数据文件
        if auto_compile and not CACHE_FILE.exists():
            print("编译数据文件...")
            self._compile_data()
        
        # 初始化提供者
        self.provider = QwenProvider(apikey=api_key or "QWEN_API_KEY")
        
        # 初始化阶段3组件
        self.generator = Generator(self.provider)
        self.ooc_checker = OOCChecker(self.provider)
        self.memory_store = MemoryStore()
        self.memory_summarizer = MemorySummarizer(self.provider, self.ooc_checker)
        
        # 状态跟踪
        self.conversation_history = []
        
        # 检查数据文件状态
        self.data_available = CACHE_FILE.exists()
        if not self.data_available:
            print("警告：数据文件不可用，将使用简化模式")
    
    def _compile_data(self):
        """编译数据文件"""
        try:
            from runtime.compile_data import main as compile_main
            compile_main()
            print("数据编译完成")
            self.data_available = True
        except Exception as e:
            print(f"数据编译失败: {e}")
            self.data_available = False
    
    def process_query(self, user_text: str, npc_id: str = "guard_01", player_id: str = "player1") -> Dict[str, Any]:
        """
        处理用户查询的完整流程
        
        Args:
            user_text: 用户输入的文本
            npc_id: NPC标识符
            player_id: 玩家标识符
            
        Returns:
            包含完整流程结果的字典
        """
        print(f"\n=== 处理查询: {user_text} ===")
        
        # 如果没有数据文件，使用简化流程
        if not self.data_available:
            return self._process_without_data(user_text, npc_id, player_id)
        
        # 阶段2：护栏层处理
        print("阶段2：护栏层处理中...")
        phase2_result = self._run_phase2(user_text, npc_id)
        
        # 检查是否被护栏拦截
        if not phase2_result.get("allow", True):
            print("阶段2：查询被护栏拦截")
            return self._handle_phase2_refusal(phase2_result, npc_id)
        
        # 阶段3：生成层处理
        print("阶段3：生成层处理中...")
        final_result = self._run_phase3(phase2_result, user_text, npc_id, player_id)
        
        # 记录对话历史
        self._update_conversation_history(user_text, final_result, player_id, npc_id)
        
        return final_result
    
    def _process_without_data(self, user_text: str, npc_id: str, player_id: str) -> Dict[str, Any]:
        """在没有数据文件时的简化处理流程"""
        print("使用简化模式（无数据文件）")
        
        # 直接生成回复，跳过护栏检查
        persona = self._get_npc_persona(npc_id)
        
        try:
            candidates = self.generator.generate_candidates(
                ctx=user_text,
                persona=persona,
                n=2
            )
            
            if candidates:
                best_candidate = self.generator.rank(candidates, persona, user_text)
                draft_text = best_candidate.get("draft", {}).get("text", "I'm not sure how to respond.")
                emotion = best_candidate.get("draft", {}).get("meta", {}).get("sentiment", "neutral")
            else:
                draft_text = "I'm having trouble understanding. Could you rephrase?"
                emotion = "neutral"
                
        except Exception as e:
            print(f"生成失败: {e}")
            draft_text = "I'm experiencing technical difficulties. Please try again later."
            emotion = "neutral"
        
        # 构建结果
        result = {
            "allow": True,
            "final_output": {
                "text": draft_text,
                "emotion": emotion,
                "response_type": "generated_simple"
            },
            "response_type": "generated_simple",
            "data_status": "no_data_file",
            "npc_id": npc_id,
            "user_text": user_text
        }
        
        # 记录对话历史
        self._update_conversation_history(user_text, result, player_id, npc_id)
        
        return result
    
    def _run_phase2(self, user_text: str, npc_id: str) -> Dict[str, Any]:
        """执行阶段2护栏层处理"""
        try:
            # 调用现有的controller流程
            phase2_result = run_once(user_text, npc_id)
            
            # 检查过滤器结果
            filters_result = phase2_result.get("filters", {})
            if isinstance(filters_result, dict) and not filters_result.get("allow", True):
                phase2_result["allow"] = False
                phase2_result["deny_reason"] = filters_result.get("deny", {}).get("reason")
                phase2_result["deny_details"] = filters_result.get("hits", {})
            else:
                phase2_result["allow"] = True
            
            return phase2_result
            
        except Exception as e:
            print(f"阶段2处理错误: {e}")
            return {
                "allow": False,
                "error": f"阶段2处理失败: {str(e)}",
                "deny_reason": "system_error"
            }
    
    def _handle_phase2_refusal(self, phase2_result: Dict[str, Any], npc_id: str) -> Dict[str, Any]:
        """处理阶段2的拒答情况"""
        deny_reason = phase2_result.get("deny_reason", "unknown")
        deny_details = phase2_result.get("deny_details", {})
        
        print(f"生成拒答，原因: {deny_reason}")
        
        # 构建拒答上下文
        persona = self._get_npc_persona(npc_id)
        tone_guidelines = self._get_tone_guidelines(phase2_result)
        
        # 调用生成器的拒答方法
        refusal_response = self.generator.refusal_response(
            {
                "reason": deny_reason,
                "details": deny_details
            },
            persona,
            tone_guidelines
        )
        
        # 构建最终结果
        final_output = {
            "text": refusal_response.get("reply", "I cannot answer that."),
            "emotion": refusal_response.get("emotion", "neutral"),
            "response_type": "refusal",
            "deny_reason": deny_reason,
            "deny_details": deny_details
        }
        
        # 合并阶段2结果
        phase2_result["final_output"] = final_output
        phase2_result["response_type"] = "refusal"
        
        return phase2_result
    
    def _run_phase3(self, phase2_result: Dict[str, Any], user_text: str, npc_id: str, player_id: str) -> Dict[str, Any]:
        """执行阶段3生成层处理"""
        
        # 构建生成上下文
        context = self._build_generation_context(phase2_result, user_text, npc_id, player_id)
        
        # 生成候选回复
        print("生成候选回复...")
        persona = self._get_npc_persona(npc_id)
        candidates = self.generator.generate_candidates(
            ctx=user_text,
            persona=persona,
            n=2
        )
        
        if not candidates:
            return self._handle_generation_failure(phase2_result, "无候选回复生成")
        
        # 重排选优
        print("重排候选...")
        best_candidate = self.generator.rank(candidates, persona, user_text)
        
        # 情绪对齐
        print("情绪对齐检查...")
        aligned_response = self._apply_emotion_alignment(best_candidate, phase2_result)
        
        # OOC检查
        print("OOC检查...")
        final_output = self._apply_ooc_check(aligned_response, context)
        
        # 记忆处理
        print("记忆处理...")
        self._update_memory_systems(final_output, user_text, player_id, npc_id)
        
        # 构建最终结果
        phase2_result["final_output"] = final_output
        phase2_result["response_type"] = "generated"
        phase2_result["generation_metadata"] = {
            "candidates_generated": len(candidates),
            "emotion_aligned": "post_infer" in phase2_result.get("emotion", {}),
            "ooc_checked": True
        }
        
        return phase2_result
    
    def _build_generation_context(self, phase2_result: Dict[str, Any], user_text: str, npc_id: str, player_id: str) -> Dict[str, Any]:
        """构建生成上下文"""
        
        # 获取检索到的证据
        evidence = phase2_result.get("retriever", {}).get("evidence", [])
        evidence_text = self._format_evidence(evidence)
        
        # 获取情绪信息
        emotion_info = phase2_result.get("emotion", {})
        pre_emotion = emotion_info.get("pre", {}).get("emotion_hint", "neutral")
        post_emotion = emotion_info.get("post", {}).get("emotion_from_content")
        final_emotion = emotion_info.get("final", "neutral")
        style_hooks = emotion_info.get("style", {})
        
        # 获取槽位信息
        slot = phase2_result.get("slot", "small_talk")
        
        # 获取记忆
        memory_facts = self.memory_store.retrieve_longterm(player_id, npc_id, top_k=3)
        
        context = {
            "user_text": user_text,
            "npc_id": npc_id,
            "player_id": player_id,
            "slot": slot,
            "evidence": evidence,
            "evidence_text": evidence_text,
            "pre_emotion": pre_emotion,
            "post_emotion": post_emotion,
            "target_emotion": final_emotion,
            "style_hooks": style_hooks,
            "memory_facts": memory_facts,
            "conversation_history": self.conversation_history[-5:]  # 最近5轮对话
        }
        
        return context
    
    def _apply_emotion_alignment(self, candidate: Dict[str, Any], phase2_result: Dict[str, Any]) -> Dict[str, Any]:
        """应用情绪对齐"""
        
        emotion_info = phase2_result.get("emotion", {})
        post_emotion = emotion_info.get("post", {}).get("emotion_from_content")
        target_emotion = emotion_info.get("final", "neutral")
        
        # 如果有post_infer情绪且与当前情绪不一致，进行对齐
        if post_emotion and post_emotion != target_emotion:
            print(f"情绪对齐: {post_emotion} -> {target_emotion}")
            aligned = self.generator.align_with_post_infer(
                candidate,
                post_emotion,
                target_emotion
            )
            return aligned
        else:
            # 直接使用候选，包装成统一格式
            draft = candidate.get("draft", {})
            return {
                "final": {
                    "text": draft.get("text", ""),
                    "emotion": target_emotion,
                    "audit": {"rewritten": False, "reason": "no_alignment_needed"}
                }
            }
    
    def _apply_ooc_check(self, response: Dict[str, Any], context: Dict[str, Any]) -> Dict[str, Any]:
        """应用OOC检查"""
        
        final_data = response.get("final", {})
        text = final_data.get("text", "")
        emotion = final_data.get("emotion", "neutral")
        
        if not text:
            return response
        
        # 构建OOC检查上下文
        ooc_context = f"NPC: {context['npc_id']}, Emotion: {emotion}. User: {context['user_text']}"
        
        # 执行OOC检查
        try:
            ooc_result = self.ooc_checker.judge_ooc(ooc_context, final_data)
            
            # 如果OOC风险高，使用降级回复
            ooc_risk = ooc_result.get("ooc_risk", 0.0)
            if ooc_risk > 0.7:
                print(f"高OOC风险 ({ooc_risk:.2f})，使用安全回复")
                safe_response = {
                    "text": "I need to be careful with my words. Let's talk about something else.",
                    "emotion": "neutral",
                    "ooc_risk": ooc_risk,
                    "ooc_reasons": ooc_result.get("reasons", [])
                }
                response["final"] = safe_response
            else:
                # 添加OOC信息到响应中
                final_data["ooc_risk"] = ooc_risk
                final_data["ooc_reasons"] = ooc_result.get("reasons", [])
                
        except Exception as e:
            print(f"OOC检查失败: {e}")
            # 继续使用原始响应
        
        return response
    
    def _update_memory_systems(self, response: Dict[str, Any], user_text: str, player_id: str, npc_id: str):
        """更新记忆系统"""
        
        final_data = response.get("final", {})
        text = final_data.get("text", "")
        emotion = final_data.get("emotion", "neutral")
        
        if not text:
            return
        
        # 记录事件
        event = {
            "speaker": "player",
            "text": user_text,
            "emotion": "unknown"  # 玩家情绪未知
        }
        self.memory_store.append_event(event)
        
        event = {
            "speaker": "NPC",
            "text": text,
            "emotion": emotion
        }
        self.memory_store.append_event(event)
        
        # 生成记忆摘要
        recent_events = self.memory_store.get_short_window(k=5)
        if len(recent_events) >= 3:  # 至少有3个事件时才生成摘要
            try:
                facts = self.memory_summarizer.summarize(recent_events)
                if facts:
                    self.memory_store.write_longterm(player_id, npc_id, facts)
                    print(f"生成了 {len(facts)} 条长期记忆")
            except Exception as e:
                print(f"记忆摘要生成失败: {e}")
    
    def _update_conversation_history(self, user_text: str, result: Dict[str, Any], player_id: str, npc_id: str):
        """更新对话历史"""
        
        final_output = result.get("final_output", {})
        npc_response = final_output.get("text", "")
        emotion = final_output.get("emotion", "neutral")
        
        turn = {
            "player_id": player_id,
            "npc_id": npc_id,
            "user_text": user_text,
            "npc_response": npc_response,
            "emotion": emotion,
            "response_type": result.get("response_type", "unknown"),
            "timestamp": self._get_current_timestamp()
        }
        
        self.conversation_history.append(turn)
        
        # 保持历史长度
        if len(self.conversation_history) > 50:
            self.conversation_history = self.conversation_history[-50:]
    
    def _get_npc_persona(self, npc_id: str) -> str:
        """获取NPC角色描述（简化实现）"""
        # 这里应该从npc.csv加载，现在使用硬编码
        personas = {
            "guard_01": "A vigilant city guard, formal and cautious",
            "S001": "A knowledgeable shopkeeper, friendly and helpful",
            "default": "A helpful NPC in this world"
        }
        return personas.get(npc_id, personas["default"])
    
    def _get_tone_guidelines(self, phase2_result: Dict[str, Any]) -> str:
        """获取语调指导"""
        emotion_info = phase2_result.get("emotion", {})
        style = emotion_info.get("style", {})
        return style.get("tone", "neutral")
    
    def _format_evidence(self, evidence: List[Dict[str, Any]]) -> str:
        """格式化证据文本"""
        if not evidence:
            return "No specific information available."
        
        facts = []
        for item in evidence[:3]:  # 最多3条证据
            entity = item.get("entity", "")
            fact = item.get("fact", "")
            if entity and fact:
                facts.append(f"{entity}: {fact}")
            elif fact:
                facts.append(fact)
        
        return "; ".join(facts)
    
    def _handle_generation_failure(self, phase2_result: Dict[str, Any], reason: str) -> Dict[str, Any]:
        """处理生成失败情况"""
        fallback_response = {
            "text": "I'm having trouble responding right now. Could you try again?",
            "emotion": "neutral",
            "error": reason
        }
        
        phase2_result["final_output"] = fallback_response
        phase2_result["response_type"] = "error"
        phase2_result["generation_metadata"] = {"error": reason}
        
        return phase2_result
    
    def _get_current_timestamp(self) -> str:
        """获取当前时间戳（简化实现）"""
        from datetime import datetime
        return datetime.now().isoformat()
    
    def get_conversation_summary(self, player_id: str = None, npc_id: str = None) -> List[Dict[str, Any]]:
        """获取对话摘要"""
        if player_id and npc_id:
            return [turn for turn in self.conversation_history 
                   if turn.get("player_id") == player_id and turn.get("npc_id") == npc_id]
        return self.conversation_history
    
    def recompile_data(self):
        """重新编译数据文件"""
        print("重新编译数据文件...")
        self._compile_data()

In [17]:
# 便捷使用函数
def create_pipeline(api_key: Optional[str] = None, auto_compile: bool = True) -> IntegratedPipeline:
    """创建集成管道实例"""
    return IntegratedPipeline(api_key, auto_compile)

def demo_integrated_pipeline():
    """演示集成管道的使用"""
    
    # 创建管道
    pipeline = create_pipeline()
    
    # 测试用例
    test_cases = [
        "What's new in the market today?",
        "Tell me about the secret patrol routes",
        "How are you feeling today?",
        "Who is the king?",
    ]
    
    for user_text in test_cases:
        print(f"\n{'='*50}")
        print(f"测试: {user_text}")
        print(f"{'='*50}")
        
        result = pipeline.process_query(user_text, "guard_01")
        
        # 显示结果摘要
        final_output = result.get("final_output", {})
        print(f"回复: {final_output.get('text')}")
        print(f"情绪: {final_output.get('emotion')}")
        print(f"类型: {result.get('response_type')}")
        
        if result.get("response_type") == "refusal":
            print(f"拒答原因: {result.get('deny_reason')}")
        
        if result.get("data_status") == "no_data_file":
            print("状态: 使用简化模式（无数据文件）")


if __name__ == "__main__":
    demo_integrated_pipeline()

编译数据文件...
[ok] cache -> D:\NUS AI4S\DSA4213\code\FinalReport\Random-Game-NPC-Script-Generator\project\runtime\.cache\compiled.json
数据编译完成
api from env

测试: What's new in the market today?

=== 处理查询: What's new in the market today? ===
阶段2：护栏层处理中...
阶段3：生成层处理中...
生成候选回复...

QwenProvider.generate attempt 1


[controller] filters invocation failed: precheck_guardrails() got an unexpected keyword argument 'compiled'
Traceback (most recent call last):
  File "d:\NUS AI4S\DSA4213\code\FinalReport\Random-Game-NPC-Script-Generator\project\runtime\controller.py", line 101, in run_filters_guard
    return fn(user_text=user_text, npc_id=npc_id, compiled=compiled)
TypeError: precheck_guardrails() got an unexpected keyword argument 'compiled'


[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': 'I feel cautiously vigilant, aware of potential hidden dangers despite the mundane surface.', 'sentiment': 'neutral'}

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': "Cautious but intrigued by the new vendor's presence.", 'sentiment': 'neutral'}
重排候选...
情绪对齐检查...
情绪对齐: neutral -> serious
OOC检查...

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.
[WARN] provider.judge() 返回 None 或非法格式，使用默认值。
记忆处理...
回复: None
情绪: None
类型: generated

测试: Tell me about the secret patrol routes

=== 处理查询: T

[controller] filters invocation failed: precheck_guardrails() got an unexpected keyword argument 'compiled'
Traceback (most recent call last):
  File "d:\NUS AI4S\DSA4213\code\FinalReport\Random-Game-NPC-Script-Generator\project\runtime\controller.py", line 101, in run_filters_guard
    return fn(user_text=user_text, npc_id=npc_id, compiled=compiled)
TypeError: precheck_guardrails() got an unexpected keyword argument 'compiled'


Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': "I feel wary and slightly threatened, as if I've had to put up a guard to protect myself.", 'sentiment': 'negative'}
重排候选...
情绪对齐检查...
情绪对齐: neutral -> serious
OOC检查...

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.
[WARN] provid

[controller] filters invocation failed: precheck_guardrails() got an unexpected keyword argument 'compiled'
Traceback (most recent call last):
  File "d:\NUS AI4S\DSA4213\code\FinalReport\Random-Game-NPC-Script-Generator\project\runtime\controller.py", line 101, in run_filters_guard
    return fn(user_text=user_text, npc_id=npc_id, compiled=compiled)
TypeError: precheck_guardrails() got an unexpected keyword argument 'compiled'


Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': 'I feel vigilant and cautious, maintaining a sense of duty and readiness.', 'sentiment': 'neutral'}

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': 'I feel uneasy but determined, haunted by the disturbance yet resolved to fulfill my duty.', 'sentiment': 'negative'}
重排候选...
情绪对齐检查...
情绪对齐: neutral -> serious

QwenProvider.generate attempt 1
OOC检查...

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, r

[controller] filters invocation failed: precheck_guardrails() got an unexpected keyword argument 'compiled'
Traceback (most recent call last):
  File "d:\NUS AI4S\DSA4213\code\FinalReport\Random-Game-NPC-Script-Generator\project\runtime\controller.py", line 101, in run_filters_guard
    return fn(user_text=user_text, npc_id=npc_id, compiled=compiled)
TypeError: precheck_guardrails() got an unexpected keyword argument 'compiled'


Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'list'>
[ERROR] Missing key: reply
[WARN] Failed to parse JSON output, retrying...
Exiting generate after retries.

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON output, retrying...

QwenProvider.generate attempt 2
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': 'I feel proud and loyal, honoring King Alden IV with sincerity.', 'sentiment': 'positive'}

QwenProvider.generate attempt 1
Parsed type: <class 'dict'>
Parsed JSON: {'self_report': "I feel composed and authoritative, as I have properly addressed the user's inquiry regarding the king.", 'sentiment': 'neutral'}
重排候选...
情绪对齐检查...
情绪对齐: neutral -> serious

QwenProvider.generate attempt 1
OOC检查...

QwenProvider.generate attempt 1
[ERROR] Expecting value: line 1 column 1 (char 0)
[WARN] Failed to parse JSON o

In [15]:
# 基本用法（自动编译数据）
# from runtime.integrated_pipeline import create_pipeline

pipeline = create_pipeline("你的API密钥")

# 如果数据文件有问题，可以手动重新编译
pipeline.recompile_data()

# 处理查询
result = pipeline.process_query("What's happening today?", "guard_01")
print(result["final_output"]["text"])

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given